In [1]:
import numpy as np
import pandas as pd
from analystcommunity import read_connection_data_warehouse as read_dw,connections_utils as cu,sheets
from analystcommunity import run_query
import google_sheets_methods

In [2]:
#todos los restaurante con pp en 2023
query = f"""
select source_id as uid, registered_date, birthday, birthday-registered_date as diff_regis_pp, extract(week from birthday) as week_birthday,
extract(month from birthday) as month_birthday, case when extract(month from birthday) in ('1','2','3') then 'Q1' 
when extract(month from birthday) in ('4','5','6') then 'Q2' 
when extract(month from birthday) in ('7','8','9') then 'Q3' 
else 'Q4' end as quarter_birthday, extract(year from birthday) as year_birthday,
registered_city as city, case when registered_city in ('BAQ','BOG','MDE') then 'Colombia'
when registered_city in ('SPO','BHZ','VCP','CWB') then 'Brasil' else 'México' end as country,
business_type 
from dpr_customer.obt_customer
where business_type in ('Restaurante')
and birthday is not null
--and extract(week from birthday) = extract(week from current_date) -1
and extract(year from birthday) = 2023
and registered_city not in ('MTY','GDL','PBC')
    """
    
users = read_dw.run_read_dwd_query(query)
base = pd.DataFrame(users)   
base.head()

,uid,registered_date,birthday,diff_regis_pp,week_birthday,month_birthday,quarter_birthday,year_birthday,city,country,business_type
0,350173959,2023-07-25,2023-07-25,0,30,7,Q3,2023,BOG,Colombia,Restaurante
1,350175156,2023-07-25,2023-07-25,0,30,7,Q3,2023,BAQ,Colombia,Restaurante
2,350085892,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,Restaurante
3,350108699,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,Restaurante
4,350114257,2023-07-24,2023-07-25,1,30,7,Q3,2023,BHZ,Brasil,Restaurante


In [3]:
#gmv de la primera semana
clientes = list(base['uid'])
params = {'clientes': clientes}
def gmv(clientes):
    query = f"""
 with birthdays as (
        SELECT
          uid,
          MIN(birthday) AS birthday,
          region_code
        FROM (
            SELECT
              distinct bc.customer_id::VARCHAR AS uid,
              min(ffg.close_date) AS birthday,
              s.site_identifier_value AS region_code
            FROM postgres_broadleaf_federate."broadleaf.blc_order" AS bo
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" AS bfg ON bfg.order_id = bo.order_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" AS bfo ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" AS ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" AS fo ON fo.order_id = bo.order_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" AS bop ON bop.order_id = bo.order_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" AS s ON s.site_id = bo.site_disc
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer" AS bc ON bc.customer_id = bo.customer_id
            LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order_type" AS fot ON fot.fb_order_type_id = fo.fb_order_type_id
            --inner join ciudad_registrada on bc.customer_id=ciudad_registrada.customer_id and s.site_identifier_value=ciudad_registrada.region_code
            WHERE fo.fb_order_status_id IN (1,6,7,8)
              AND bo.order_status = 'SUBMITTED'
              AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
              AND bop.archived = 'N'
              AND (fot.name IS NULL OR fot.name <> 'REFUND')
            GROUP BY bo.order_id, ffg.close_date, bc.customer_id, bc.external_id, s.site_identifier_value) AS a
        GROUP BY 1,3
        ),
        
    orders as
        (SELECT bo.customer_id::varchar as uid, s.site_identifier_value as region_code, bo.order_id, 
            bo.submit_date AT TIME ZONE 'UTC' at time zone 'america/bogota' as submit_date, ffg.close_date,
            CASE WHEN fc.business_type_id = 1 then 'Restaurante' WHEN fc.business_type_id = 2 then 'Comercio'
                WHEN fc.business_type_id = 3 then 'Hogar' WHEN fc.business_type_id = 4 then 'Institucional'
                WHEN fc.business_type_id = 5 then 'Fresco' WHEN fc.business_type_id = 6 then 'Donaciones'
                ELSE 'Null' end AS segmento, bo.order_subtotal+/*coalesce(fo.total_tax_iva,0)*/coalesce(bo.total_tax,0)+coalesce(bo.total_shipping,0) AS gmv_moneda_local,
            CASE WHEN s.site_identifier_value in ('BAQ','BOG','MDE') then (bo.order_subtotal+coalesce(bo.total_tax,0)+coalesce(bo.total_shipping,0))/3776.0
                WHEN s.site_identifier_value in ('CMX','GDL','PBC','MTY') then (bo.order_subtotal+coalesce(bo.total_tax,0)+coalesce(bo.total_shipping,0))/19.65
                WHEN s.site_identifier_value in ('SPO','BHZ','CWB','VCP') then (bo.order_subtotal+coalesce(bo.total_tax,0)+coalesce(bo.total_shipping,0))/4.75 end AS gmv_usd
        FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg ON bfg.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo ON fo.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop ON bop.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s ON s.site_id = bo.site_disc --------VA CON bo
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer" bc ON bc.customer_id = bo.customer_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order_type" fot ON fot.fb_order_type_id=fo.fb_order_type_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_customer" fc ON fc.customer_id = bc.customer_id-------VA CON bc
        WHERE fo.fb_order_status_id IN (1,6,7,8) AND bo.order_status = 'SUBMITTED' 
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED') AND bop.archived='N' AND (fot.name IS NULL OR fot.name<>'REFUND') 
        AND submit_date>'2020-06-01' --AND s.site_identifier_value in ('BOG','BAQ','CMX','SPO')
        GROUP BY bo.customer_id,s.site_identifier_value,bo.order_id,bo.submit_date,ffg.close_date,fc.business_type_id,
        bo.order_subtotal,bo.total_tax,bo.total_shipping)

    SELECT a.uid, a.region_code as city,
    date_part('year',close_date)||'-'||RIGHT('0'||date_part('week',close_date),2) as close_week, date_part('year',birthday)||'-'||RIGHT('0'||date_part('week',birthday),2) as birth_week,
    sum(gmv_moneda_local) as gmv_local, 
    sum(gmv_usd) as gmv_usd
    FROM orders a
    left join birthdays b on b.uid = a.uid
    WHERE a.uid in {repr(tuple(map(str, clientes)))} and close_week=birth_week
    --close_date>='2022-01-01' AND close_date<=date(GETDATE() AT TIME ZONE 'UTC' AT TIME ZONE 'America/Bogota')
    GROUP BY 1,2,3,4

        """
    df = read_dw.run_read_prod_query(query)
    return df
gmv_summary = gmv(clientes)
#gmv_summary.head()
gmv_summary

,uid,city,close_week,birth_week,gmv_local,gmv_usd
0,344199649,SPO,2023-22,2023-22,108.23000,22.7852631000
1,237621986,SPO,2023-22,2023-22,184.73000,38.8905263000
2,342775807,BAQ,2023-18,2023-18,121417.07000,32.1549443855
3,346652979,BAQ,2023-24,2023-24,438315.69000,116.0793670550
4,334238592,SPO,2023-15,2023-15,366.00000,77.0526315000
...,...,...,...,...,...,...
29276,200098738,MDE,2023-24,2023-24,54193.28000,14.3520338983
29277,281448484,MDE,2023-01,2023-01,607401.06000,160.8583315677
29278,318021448,MDE,2023-11,2023-11,109612.59000,29.0287579449
29279,346463394,MDE,2023-24,2023-24,185713.49000,49.1825979872


In [4]:
#gmv del primer pedido
clientes = list(base['uid'])
params = {'clientes': clientes}
def gmv_pp(clientes):
    query_pp = f"""
 with base as (
SELECT
    dcus.source_id AS customer_id,
    fs.order_close_date,
    sum(fs.gmv) as gmv, sum(fs.gmv_local) as gmv_local
 
FROM dpr_sales.fact_sales AS fs
INNER JOIN dpr_shared.dim_customer AS dcus ON (fs.dim_customer = dcus.customer_id)
INNER JOIN dpr_shared.dim_site AS ds ON (fs.dim_site = ds.site_id)
INNER JOIN dpr_shared.dim_business_segment AS dbs ON (fs.dim_business_segment = dbs.business_segment_id)

WHERE fs.gmv_enabled = true
    AND dcus.archived = false
    --AND dcus.is_kam = false
    AND fs.dim_status = 1 and fs.is_deleted=false
    --AND dbs.description = 'Restaurante'
    and extract(year from fs.order_close_date) in ('2023')
 group by 1,fs.order_close_date),
 
base1 as(select distinct customer_id, sum(gmv) as gmv,sum(gmv_local) as gmv_local, order_close_date,ROW_NUMBER () OVER (partition by customer_id ORDER BY order_close_date asc) as row
 from base
 group by 1,4)
 
 select distinct customer_id as uid, gmv as gmv_pp,gmv_local as gmv_pp_local
 from base1
 where row=1
 and customer_id in {repr(tuple(map(str, clientes)))} 
   
        """
    df_pp = read_dw.run_read_dwd_query(query_pp)
    return df_pp
gmv_sum_pp = gmv_pp(clientes)
gmv_sum_pp

,uid,gmv_pp,gmv_pp_local
0,163912195,273.15604,1031439.20000
1,21592891,448.64247,1694077.22000
2,196452737,29.70524,141.10000
3,284180643,23.89232,90217.71000
4,284578674,35.36000,167.96000
...,...,...,...
29265,345913204,25.35366,120.43000
29266,346889476,25.22513,119.81960
29267,348430171,21.75156,103.32000
29268,349664231,55.11949,208131.68000


In [5]:
#merge
gmv_summary['uid'] = pd.to_numeric(gmv_summary['uid'], errors='coerce').astype(int)
gmv_sum_pp['uid'] = pd.to_numeric(gmv_sum_pp['uid'], errors='coerce').astype(int)
merged_d = pd.merge(base, gmv_summary[['uid', 'gmv_local','gmv_usd']], on=['uid'], how='left')
merged_d = merged_d.fillna(0)
merged_d = pd.merge(merged_d, gmv_sum_pp[['uid', 'gmv_pp_local','gmv_pp']], on=['uid'], how='left')
merged_d = merged_d.fillna(0)
merged_d.registered_date = merged_d.registered_date.astype(str)
merged_d.birthday = merged_d.birthday.astype(str)
merged_d['gmv_local']=pd.to_numeric(merged_d['gmv_local'],errors='coerce')
merged_d['gmv_usd']=pd.to_numeric(merged_d['gmv_usd'],errors='coerce')
merged_d['gmv_pp_local']=pd.to_numeric(merged_d['gmv_pp_local'],errors='coerce')
merged_d['gmv_pp']=pd.to_numeric(merged_d['gmv_pp'],errors='coerce')
merged_d.head()

,uid,registered_date,birthday,diff_regis_pp,week_birthday,month_birthday,quarter_birthday,year_birthday,city,country,business_type,gmv_local,gmv_usd,gmv_pp_local,gmv_pp
0,350173959,2023-07-25,2023-07-25,0,30,7,Q3,2023,BOG,Colombia,Restaurante,218571.02,57.884274,218571.020,57.88413
1,350175156,2023-07-25,2023-07-25,0,30,7,Q3,2023,BAQ,Colombia,Restaurante,142454.77,37.726369,142454.772,37.72617
2,350085892,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,Restaurante,621.62,130.867368,621.620,130.86735
3,350108699,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,Restaurante,541.20,113.936842,541.200,113.93682
4,350114257,2023-07-24,2023-07-25,1,30,7,Q3,2023,BHZ,Brasil,Restaurante,247.18,52.037895,247.180,52.03789


In [6]:
#importar archivo de inputs
from procurement_lib import GoogleSheet
gs = GoogleSheet("1lHM0r0bXHkoN5w7_AaE-92oUitzCXc0_xSvKukyTXPY") #Id de tu sheets 
df_input = gs.get_as_dataframe("Base") #Nombre de tu hoja.
merged_d['Inputs'] = merged_d['uid'].isin(df_input['uid']).map({True: 'Si', False: 'No'})
merged_d.head()

,uid,registered_date,birthday,diff_regis_pp,week_birthday,month_birthday,quarter_birthday,year_birthday,city,country,business_type,gmv_local,gmv_usd,gmv_pp_local,gmv_pp,Inputs
0,350173959,2023-07-25,2023-07-25,0,30,7,Q3,2023,BOG,Colombia,Restaurante,218571.02,57.884274,218571.020,57.88413,No
1,350175156,2023-07-25,2023-07-25,0,30,7,Q3,2023,BAQ,Colombia,Restaurante,142454.77,37.726369,142454.772,37.72617,No
2,350085892,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,Restaurante,621.62,130.867368,621.620,130.86735,Si
3,350108699,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,Restaurante,541.20,113.936842,541.200,113.93682,Si
4,350114257,2023-07-24,2023-07-25,1,30,7,Q3,2023,BHZ,Brasil,Restaurante,247.18,52.037895,247.180,52.03789,Si


In [7]:
#queries encuesta
from procurement_lib import redash
df_bra=redash.get_query_results(166752)
df_col=redash.get_query_results(166744)
df_mex=redash.get_query_results(166748)
df_encuesta = pd.concat([df_bra, df_col, df_mex], ignore_index=True)
df_encuesta.head()

,first_commercial,hunter_name,hunter_email,customer_id,customer_name,customer_email,order_id,week,year,program_day,...,categorias,fb_order_status,supervisor,segmento,city,update_as,arpu_mxn,fecha_visita,hunter_encuesta,email_encuesta
0,20255,Adonys John Souza Araujo,hunter01@frubana.com,21102617,juninhos restaurante,marcioaparecido_miranda@hotmail.com,18257478,1,2023,2023-01-06T15:52:08.004,...,2.0,FINALIZED,None,Restaurante,SPO,2023-07-25T21:30:37.716,661,None,None,None
1,22950,Thiago Kotwica Prendes Hevia,hunter42@frubana.com,21200722,Brasil Prime,sjeovani72@gmail.com,23044542,29,2023,2023-07-19T16:30:00.924,...,2.0,FINALIZED,None,Restaurante,SPO,2023-07-25T21:30:37.716,256,None,None,None
2,20001,EDUARDO MARCOS AGUIAR JUNIOR,hunter09@frubana.com,21476442,RESTAURANTE NOVO TBC LTDA,novotbc@gmail.com,20357553,13,2023,2023-03-29T15:46:40.854,...,1.0,FINALIZED,None,Restaurante,SPO,2023-07-25T21:30:37.716,136,None,None,None
3,23753,Marcio Romualdo,hunter52@frubana.com,22104609,Burger - Vila Madalena,vilamadalena@busger.com.br,22776721,28,2023,2023-07-10T03:55:30.704,...,1.0,FINALIZED,None,Restaurante,SPO,2023-07-25T21:30:37.716,1315,None,None,None
4,23600,Cristiane Gravinez,hunter46@frubana.com,22752686,Lanchonete Estrela da Brigadeiro,cezar.l.novais@gmail.com,20950080,17,2023,2023-04-24T15:21:55.753,...,1.0,FINALIZED,None,Restaurante,SPO,2023-07-25T21:30:37.716,269,None,None,None


In [8]:
merged_d['Encuesta'] = merged_d['uid'].isin(df_encuesta['customer_id']).map({True: 'Si', False: 'No'})

In [9]:
#importar archivos hunters comex
from procurement_lib import GoogleSheet
gs = GoogleSheet("1ocJ1NxVuO0aAQq1hd8Bn4N-dDe1y2RXKYtyBubLAM6U") #Id de tu sheets 
df_paul = gs.get_as_dataframe("Paul / -295") #Nombre de tu hoja.
df_paul["Hunter"]= "Paul"
df_paul=df_paul[['User_ID','Hunter']]

df_yiseth = gs.get_as_dataframe("Yiseth Parra") #Nombre de tu hoja.
df_yiseth["Hunter"]= "Yiseth"
df_yiseth=df_yiseth[['User_ID','Hunter']]

df_yise = gs.get_as_dataframe("Yise / -243") #Nombre de tu hoja.
df_yise["Hunter"]= "Yise"
df_yise=df_yise[['User_ID','Hunter']]

df_william = gs.get_as_dataframe("William") #Nombre de tu hoja.
df_william["Hunter"]= "William"
df_william=df_william[['User_ID','Hunter']]

df_julieta = gs.get_as_dataframe("Julieta 5") #Nombre de tu hoja.
df_julieta["Hunter"]= "Julieta"
df_julieta=df_julieta[['User_ID','Hunter']]

df_sara = gs.get_as_dataframe("Sara / 4") #Nombre de tu hoja.
df_sara["Hunter"]= "Sara"
df_sara=df_sara[['User_ID','Hunter']]

df_maye = gs.get_as_dataframe("MAYE/5") #Nombre de tu hoja.
df_maye["Hunter"]= "Maye"
df_maye=df_maye[['User_ID','Hunter']]

df_julio = gs.get_as_dataframe("Julio / 8002") #Nombre de tu hoja.
df_julio["Hunter"]= "Julio"
df_julio=df_julio[['User_ID','Hunter']]

df_samir = gs.get_as_dataframe("Samir / 52157") #Nombre de tu hoja.
df_samir["Hunter"]= "Samir"
df_samir=df_samir[['User_ID','Hunter']]

df_dahiana = gs.get_as_dataframe("Dahiana / 1") #Nombre de tu hoja.
df_dahiana["Hunter"]= "Dahiana"
df_dahiana=df_dahiana[['User_ID','Hunter']]

In [10]:
#importar archivos hunters brasil
gs_bra = GoogleSheet("1o_OQ9UK0BtnyZcZdFfHTwoZcZ88y6idI_6oZxbJA19w") #Id de tu sheets 

df_gabriel = gs_bra.get_as_dataframe("Gabriel 6") #Nombre de tu hoja.
df_gabriel["Hunter"]= "Gabriel"
df_gabriel=df_gabriel[['User_ID','Hunter']]

df_priscila = gs_bra.get_as_dataframe("Priscila 7") #Nombre de tu hoja.
df_priscila["Hunter"]= "Priscila"
df_priscila=df_priscila[['User_ID','Hunter']]

df_fernanda = gs_bra.get_as_dataframe("Fernanda 5") #Nombre de tu hoja.
df_fernanda["Hunter"]= "Fernanda"
df_fernanda=df_fernanda[['User_ID','Hunter']]

df_tania = gs_bra.get_as_dataframe("Tania 8") #Nombre de tu hoja.
df_tania["Hunter"]= "Tania"
df_tania=df_tania[['User_ID','Hunter']]

In [11]:
hunters = pd.concat([df_paul, df_yiseth, df_yise, df_william, df_julieta, df_sara, df_maye, df_julio, df_samir, df_dahiana, df_gabriel,
                  df_priscila, df_fernanda, df_tania], ignore_index=True)

def remove_slash(value):
    if isinstance(value, str) and '/' in value:
        return value.replace('/', '')
    return value

hunters['User_ID'] = hunters['User_ID'].apply(remove_slash)
hunters = hunters.dropna()
hunters = hunters.drop_duplicates(subset=['User_ID'])
hunters['User_ID'] = pd.to_numeric(hunters['User_ID'], errors='coerce').astype(int)
hunters.rename(columns={'User_ID': 'uid'}, inplace=True)
df_final = pd.merge(merged_d, hunters, on=['uid'], how='left')
df_final['Hunter'] = df_final['Hunter'].fillna('')
df_final.head()

,uid,registered_date,birthday,diff_regis_pp,week_birthday,month_birthday,quarter_birthday,year_birthday,city,country,business_type,gmv_local,gmv_usd,gmv_pp_local,gmv_pp,Inputs,Encuesta,Hunter
0,350173959,2023-07-25,2023-07-25,0,30,7,Q3,2023,BOG,Colombia,Restaurante,218571.02,57.884274,218571.020,57.88413,No,Si,
1,350175156,2023-07-25,2023-07-25,0,30,7,Q3,2023,BAQ,Colombia,Restaurante,142454.77,37.726369,142454.772,37.72617,No,Si,
2,350085892,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,Restaurante,621.62,130.867368,621.620,130.86735,Si,No,
3,350108699,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,Restaurante,541.20,113.936842,541.200,113.93682,Si,No,
4,350114257,2023-07-24,2023-07-25,1,30,7,Q3,2023,BHZ,Brasil,Restaurante,247.18,52.037895,247.180,52.03789,Si,Si,


In [12]:
#query frubonos utilizados
query2 = f"""
with orders as
    (SELECT bo.customer_id::varchar as uid, s.site_identifier_value as region_code, bo.order_id, 
        date(bo.submit_date  AT TIME ZONE 'UTC' at time zone 'america/bogota') as submit_date,ffg.close_date
    FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg ON bfg.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo ON fo.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop ON bop.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s ON s.site_id = bo.site_disc --------VA CON bo
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer" bc ON bc.customer_id = bo.customer_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order_type" fot ON fot.fb_order_type_id=fo.fb_order_type_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_customer" fc ON fc.customer_id = bc.customer_id-------VA CON bc
    WHERE fo.fb_order_status_id IN (1,6,7,8) AND bo.order_status = 'SUBMITTED' 
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED') AND bop.archived='N' AND (fot.name IS NULL OR fot.name<>'REFUND')
    -- AND submit_date>'2020-06-01'
    GROUP BY bo.customer_id,s.site_identifier_value,bo.order_id,bo.submit_date,ffg.close_date,fc.business_type_id),
    
    birthday as ((SELECT uid, region_code, min(close_date) as bday 
        FROM orders 
            GROUP BY 1,2))
            
        select 
        distinct broadleaf.customer_id as uid,bday,extract(week from bday) as bweek,--broadleaf.week,broadleaf.close_date,broadleaf.submit_date,
        broadleaf.site,
        broadleaf.reason,broadleaf.reference,
        sum(broadleaf.frubonos_redimidos) AS frubonos_local,
         CASE WHEN broadleaf.site in ('BAQ','BOG','MDE') then sum(broadleaf.frubonos_redimidos)/3776.0
                WHEN broadleaf.site in ('CMX','GDL','PBC','MTY') then sum(broadleaf.frubonos_redimidos)/19.65
                WHEN broadleaf.site in ('SPO','BHZ','CWB','VCP') then sum(broadleaf.frubonos_redimidos)/4.75 end AS frubonos_usd
        FROM (
            SELECT 
            to_char(broadleaf.close_date, 'IYYY-IW') as week,broadleaf.close_date,broadleaf.submit_date,
            broadleaf.site,broadleaf.customer_id,bday,
            (
                SELECT reason
                FROM postgres_growth."growth_checkout_wallet_database.fund_transaction" ff
                INNER JOIN postgres_growth."growth_checkout_wallet_database.wallet_transaction"
                wtt ON wtt.id = ff.wallet_transaction_id
                AND wtt.parent_id is null
                WHERE ff.fund_id = ft.fund_id
                AND ff.type = 'CREDIT'
            ) reason,
            (
                SELECT reference
                FROM postgres_growth."growth_checkout_wallet_database.fund_transaction" ff
                INNER JOIN postgres_growth."growth_checkout_wallet_database.wallet_transaction"
                wtt ON wtt.id = ff.wallet_transaction_id
                AND wtt.parent_id is null
                WHERE ff.fund_id = ft.fund_id
                AND ff.type = 'CREDIT'
            ) reference,
            ft.amount - coalesce(
                (
                    SELECT sum(ftx.amount)
                    FROM postgres_growth."growth_checkout_wallet_database.wallet_transaction" w
                    INNER JOIN postgres_growth."growth_checkout_wallet_database.fund_transaction" ftx 
                    ON ftx.wallet_transaction_id = w.id AND ftx.fund_id = ft.fund_id
                    WHERE w.parent_id= wt.id and w.type = 'CREDIT'),0
            ) frubonos_redimidos
            FROM (
                SELECT 
                bo.order_id,
                MAX(bs.site_identifier_value) AS site,
                MAX(bo.customer_id) AS customer_id,
                MAX(bo.email_address) AS email_address,
                MAX(bo.order_number) AS order_number,
                MAX(bo.order_total) AS order_total,
                MAX(ffg.close_date) AS close_date,
                MAX(bo.submit_date) AS submit_date
                FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
                INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" bs ON bo.site_disc = bs.site_id
                INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg ON bfg.order_id = bo.order_id
                INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
                WHERE bo.order_status = 'SUBMITTED'
                AND ffg.close_date  >= '2023-01-01' 
                AND ffg.close_date <= CURRENT_DATE
                GROUP BY 1
            ) broadleaf
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop 
            ON bop.order_id = broadleaf.order_id AND bop.archived = 'N'
            INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_payment" fop ON fop.order_payment_id = bop.order_payment_id
            INNER JOIN postgres_growth."growth_payments.payment_transaction" pt ON pt.token = fop.payment_transaction_token
            INNER JOIN postgres_growth."growth_checkout_wallet_database.wallet_transaction" wt ON wt.uuid = pt.provider_payment_id
            INNER JOIN postgres_growth."growth_checkout_wallet_database.fund_transaction" ft ON ft.wallet_transaction_id = wt.id
            left join birthday b on b.uid=broadleaf.customer_id
            WHERE bop.payment_type = 'CUSTOMER_CREDIT_FRUBANA'
        ) as broadleaf
        where --broadleaf.customer_id in ('134625559')
        broadleaf.reason IN ('CORTESIA','MARKETING')
        and broadleaf.reference ILIKE '%adq%' or broadleaf.reference ILIKE  'acq%'
        and broadleaf.frubonos_redimidos>0
        and broadleaf.close_date-bday between 0 and 7
        group by 1,2,3,4,5,6
   -- )


     /*   SELECT week AS close_week, site AS region_code,
        SUM(CASE WHEN reason='MARKETING' THEN monto_frubonos_local ELSE 0 END) AS monto_frubonos_local_mkt,
        SUM(CASE WHEN reason='CORTESIA' THEN monto_frubonos_local ELSE 0 END) AS monto_frubonos_local_cortesia
        FROM all_frubonos WHERE reason IN ('CORTESIA','MARKETING')
        GROUP BY 1,2*/
 """
df_frubonos = read_dw.run_read_prod_query(query2)

In [13]:
df_final = df_final.merge(df_frubonos[['uid', 'frubonos_local','frubonos_usd']], on='uid', how='left')
df_final['frubonos_local'] = df_final['frubonos_local'].fillna(0)
df_final['frubonos_usd'] = df_final['frubonos_usd'].fillna(0)
df_final['frubonos_local']=pd.to_numeric(df_final['frubonos_local'],errors='coerce')
df_final['frubonos_usd']=pd.to_numeric(df_final['frubonos_usd'],errors='coerce')

In [14]:
#query frubonos recibidos
import http.client
import json
import time
import time
from datetime import datetime, date, timedelta
import pandas as pd
def solicitar_query_as_df(api_key, query_id):
    h1 = http.client.HTTPSConnection('internal-redash.federate.frubana.com')
    h1.request('POST', '/api/queries/{query_id}/results'.format(query_id=query_id), body=json.dumps({'max_age': 0}), headers={'Authorization':api_key})
    jobs_response = h1.getresponse()
    jobs_bytes = jobs_response.read()
    status = json.loads(jobs_bytes)['job']['status']
    id_jobs = json.loads(jobs_bytes)['job']['id']
    time.sleep(30)
    while status in [1, 2]:
        h1.request('GET', f'/api/jobs/{id_jobs}', body=None, headers={'Authorization':api_key})
        jobs_in_progess = h1.getresponse()
        jobs_bytes_in_progess = jobs_in_progess.read()
        status = json.loads(jobs_bytes_in_progess)['job']['status']
        print('query_executing...')
        time.sleep(30)
    h1.request('GET', '/api/queries/{query_id}/results'.format(query_id=query_id), body=None, headers={'Authorization':api_key})
    query_response = h1.getresponse()
    a = query_response.read()
    df = pd.DataFrame(json.loads(a)['query_result']['data']['rows'])
    return df

api_key_redash = 'LBOPW8dZ6fSxiiRdHzUVsqU3C1G0YaUWDi7PaK0O'
df_fru = solicitar_query_as_df(api_key=api_key_redash, query_id=172239)
df_fru

query_executing...


,customer_id,balance,reason,reference,subreason,frubonos,frubonos_usd,type,created_by,city,created_by1,date_created
0,-252748,10000.0,CORTESIA,ACQUISITION,FIRST_ORDERS,15000.0,3.972458,CREDIT,BROADLEAF,BOG,UNKNOWN,2023-02-10T16:53:53.851
1,-239484,0.0,CORTESIA,ACQUISITION,FIRST_ORDERS,45000.0,11.917373,CREDIT,BROADLEAF,BOG,UNKNOWN,2023-02-21T15:08:52.718
2,15997344,0.0,MARKETING,adquisicion,ACQUISITION,75000.0,19.862288,CREDIT,UNKNOWN,BOG,UNKNOWN,2023-01-03T12:19:20.213
3,20830491,30.0,MARKETING,ACQUISITION,ACQUISITION,15.0,3.157895,CREDIT,UNKNOWN,SPO,UNKNOWN,2023-07-19T22:08:40.094
4,30717630,0.0,MARKETING,adquisicion,ACQUISITION,250.0,52.631579,CREDIT,UNKNOWN,SPO,UNKNOWN,2023-01-17T12:42:10.178
...,...,...,...,...,...,...,...,...,...,...,...,...
38827,349661841,50.0,MARKETING,ACQUISITION,ACQUISITION,50.0,2.544529,CREDIT,UNKNOWN,CMX,UNKNOWN,2023-07-24T17:19:00.301
38828,349664453,50.0,MARKETING,ACQUISITION,ACQUISITION,50.0,2.544529,CREDIT,UNKNOWN,CMX,UNKNOWN,2023-07-24T17:19:01.219
38829,349665274,50.0,MARKETING,ACQUISITION,ACQUISITION,50.0,2.544529,CREDIT,UNKNOWN,CMX,UNKNOWN,2023-07-24T17:19:01.888
38830,349668112,50.0,MARKETING,ACQUISITION,ACQUISITION,50.0,2.544529,CREDIT,UNKNOWN,CMX,UNKNOWN,2023-07-24T17:19:01.373


In [15]:
#query campañas
query3 = f"""
select
distinct c.customer_id,
c.status,
s.reward, c.rewards_subreason

from postgres_growth."growth_gamification.campaign" c
LEFT JOIN postgres_growth."growth_gamification.campaign_site" cs ON cs.campaign_id = c.id
LEFT JOIN postgres_growth."growth_gamification.step" s ON s.campaign_id = c.id
where --date(c.created_at)>'2023-01-01'
c.rewards_subreason in ('ACQUISITION')
 """
df_campaign = read_dw.run_read_prod_query(query3)

In [16]:
df_final['Campaña/Frubonos'] = np.where(
    (df_final['uid'].isin(df_fru['customer_id'])) | (df_final['uid'].isin(df_campaign['customer_id'])),
    'Si',
    'No')
df_final.head()

,uid,registered_date,birthday,diff_regis_pp,week_birthday,month_birthday,quarter_birthday,year_birthday,city,country,...,gmv_local,gmv_usd,gmv_pp_local,gmv_pp,Inputs,Encuesta,Hunter,frubonos_local,frubonos_usd,Campaña/Frubonos
0,350173959,2023-07-25,2023-07-25,0,30,7,Q3,2023,BOG,Colombia,...,218571.02,57.884274,218571.020,57.88413,No,Si,,0.0,0.0,No
1,350175156,2023-07-25,2023-07-25,0,30,7,Q3,2023,BAQ,Colombia,...,142454.77,37.726369,142454.772,37.72617,No,Si,,0.0,0.0,No
2,350085892,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,...,621.62,130.867368,621.620,130.86735,Si,No,,0.0,0.0,No
3,350108699,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,...,541.20,113.936842,541.200,113.93682,Si,No,,0.0,0.0,No
4,350114257,2023-07-24,2023-07-25,1,30,7,Q3,2023,BHZ,Brasil,...,247.18,52.037895,247.180,52.03789,Si,Si,,0.0,0.0,No


In [17]:
#retención 6 semanas
clientes = list(df_final['uid'])
params = {'clientes': clientes}
def ret(clientes):
    query_ret = f"""
with bd_users as (
 SELECT 
            distinct bc.customer_id AS customer_id,
            s.site_identifier_value AS city,
            min(ffg.close_date) as birthday,
            to_char(min(ffg.close_date), 'YYYY-IW') as week_bd
            FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg ON bfg.order_id = bo.order_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo ON fo.order_id = bo.order_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop ON bop.order_id = bo.order_id
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s ON s.site_id = bo.site_disc
            INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer" bc ON bc.customer_id = bo.customer_id
            LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot     ON fot.fb_order_type_id=fo.fb_order_type_id
            LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_customer" fc ON fc.customer_id = bo.customer_id
                WHERE fo.fb_order_status_id IN (1,6,7,8)
                AND bo.order_status = 'SUBMITTED'
                AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
                AND bop.archived = 'N'
                --AND business_type_id = 1 -- filtro solo por RESTAURANTES
                AND (fot.name IS NULL OR (fot.name <> 'REFUND'and fot.name <> 'COURTESY'))
            GROUP BY 1,2
           -- having to_char(birthday,'YYYY-IW') >= TO_CHAR(current_date-133,'YYYY-IW')
           ),
            
    gmv_users as (select 
customer_id,
a.region_code, 
to_char(close_date, 'YYYY-IW') as week,
min(close_date) as min_date_week,
sum(gmv) as gmv_lc,
sum(case when a.region_code in ('CMX','GDL','PBC') then gmv/19.65
         when a.region_code IN ('SPO','BHZ','VCP','CWB') then gmv/4.75
         else gmv/3776 end) as gmv_usd
from (
    SELECT
		ffg.close_date,
		bo.submit_date
		,bo.customer_id
		,s.site_identifier_value AS region_code
		,sum(oipd.quantity * foi.step_unit * boi.sale_price /*boi.retail_price replaced automatically for boi.sale_price*/ + coalesce(fo.total_tax_iva,0))::integer as gmv
        ,bo.order_id
	FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item" bfgi
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order" bo ON bo.order_id = bfg.order_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s ON bo.site_disc = s.site_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item" boi	ON bfgi.order_item_id = boi.order_item_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item" foi 	ON boi.order_item_id = foi.order_item_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo ON fo.order_id = bo.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order_type" fot ON fot.fb_order_type_id = fo.fb_order_type_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl" oipd ON oipd.order_item_id = boi.order_item_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_customer" fc ON fc.customer_id = bo.customer_id
    
	WHERE  bo.order_status = 'SUBMITTED'
		AND fo.fb_order_status_id IN (1,6,7,8)
		AND (fot.name IS NULL OR fot.name <> 'REFUND')
		and to_char(close_date,'YYYY') = 2023
		--AND business_type_id = 1
	GROUP BY 1,2,3,4,6
	) as a

group by 1,2,3)
    
select
a.city,
a.customer_id as uid,
((min_date_week-EXTRACT(DAYOFWEEK from min_date_week)+1) - (birthday-EXTRACT(DAYOFWEEK from birthday)+1))/7 as week_cohort,
a.week_bd,
b.week,
sum(gmv_usd) as gmv_usd, sum(gmv_lc) as gmv_local
from bd_users a
left join gmv_users b on a.customer_id=b.customer_id

where a.customer_id in {repr(tuple(map(str, clientes)))}
and substring(week, 6,2) - substring(week_bd, 6,2) between 1 and 6
group by 1,2,3,4,5

        """
    df = read_dw.run_read_prod_query(query_ret)
    return df
df_retention = ret(clientes)
#gmv_summary.head()
df_retention

,city,uid,week_cohort,week_bd,week,gmv_usd,gmv_local
0,BAQ,338733528,4,2023-16,2023-20,80.0000,302746
1,CMX,295098132,4,2023-06,2023-10,36.6412,720
2,SPO,331513113,3,2023-14,2023-17,26.1052,124
3,SPO,336885770,2,2023-15,2023-17,34.1052,162
4,SPO,346384915,1,2023-24,2023-25,26.7368,127
...,...,...,...,...,...,...,...
68750,BOG,339319987,5,2023-16,2023-21,249.0000,944558
68751,SPO,302197188,5,2023-07,2023-12,38.7368,184
68752,SPO,302197188,3,2023-07,2023-10,33.6841,160
68753,BOG,326975355,4,2023-13,2023-17,82.0000,311392


In [18]:
# Utilizar pivot_table para transformar as semanas em colunas
df_pivot = df_retention.pivot_table(index='uid', columns='week_cohort', values='gmv_usd', aggfunc='sum')

df_pivot.reset_index(inplace=True)
df_pivot

df_final = df_final.merge(df_pivot, on='uid', how='left')

df_final = df_final.fillna(0)
df_final.columns = df_final.columns.map(lambda x: f'W{x}' if isinstance(x, int) and x <= 6 else x)
df_final['W1'] = df_final['W1'].astype(float)
df_final['W2']=df_final['W2'].astype(float)
df_final['W3']=df_final['W3'].astype(float)
df_final['W4']=df_final['W4'].astype(float)
df_final['W5']=df_final['W5'].astype(float)
df_final['W6']=df_final['W6'].astype(float)
df_final.head()


,uid,registered_date,birthday,diff_regis_pp,week_birthday,month_birthday,quarter_birthday,year_birthday,city,country,...,Hunter,frubonos_local,frubonos_usd,Campaña/Frubonos,W1,W2,W3,W4,W5,W6
0,350173959,2023-07-25,2023-07-25,0,30,7,Q3,2023,BOG,Colombia,...,,0.0,0.0,No,0.0,0.0,0.0,0.0,0.0,0.0
1,350175156,2023-07-25,2023-07-25,0,30,7,Q3,2023,BAQ,Colombia,...,,0.0,0.0,No,0.0,0.0,0.0,0.0,0.0,0.0
2,350085892,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,...,,0.0,0.0,No,0.0,0.0,0.0,0.0,0.0,0.0
3,350108699,2023-07-24,2023-07-25,1,30,7,Q3,2023,VCP,Brasil,...,,0.0,0.0,No,0.0,0.0,0.0,0.0,0.0,0.0
4,350114257,2023-07-24,2023-07-25,1,30,7,Q3,2023,BHZ,Brasil,...,,0.0,0.0,No,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
content=[df_final.columns.values.tolist()]+df_final.values.tolist()
ID_PERFORMANCE_DASH_DATA = '1VlM-Cyd9_dr-VPOko7GccgeaalNc4aq4hnNtWZr5Byg'

google_sheets_methods.clear_content(ID_PERFORMANCE_DASH_DATA,'base','A:AA')
google_sheets_methods.write_content(ID_PERFORMANCE_DASH_DATA,'base','A:AA',content)
print("New budgets updated")

Range successfully deleted
File successfully updated
New budgets updated


In [20]:
agregacoes = {
    'uid': 'count',                  # Contagem de IDs
    'gmv_usd': 'sum',                # Soma de GMV
    'gmv_pp': 'sum',                 # Soma de GMV_pp
    'frubonos_usd': 'sum',           # Soma de frubonos_usd
    'W1': lambda x: (x > 0).sum(),    # Contagem de clientes com valor maior que 0 na coluna W1
    'W2': lambda x: (x > 0).sum(),
    'W3': lambda x: (x > 0).sum(),
    'W4': lambda x: (x > 0).sum(),
    'W5': lambda x: (x > 0).sum(),
    'W6': lambda x: (x > 0).sum()
}

grouped_df = df_final.groupby(['week_birthday', 'month_birthday', 'city', 'country', 'business_type']).agg(agregacoes).reset_index()
grouped_df['arpu']=grouped_df['gmv_pp']/grouped_df['uid']
grouped_df.rename(columns={'uid': 'cant_users'}, inplace=True)
grouped_df.rename(columns={'gmv_usd': 'gmv'}, inplace=True)
grouped_df.rename(columns={'frubonos_usd': 'Frubonos'}, inplace=True)
grouped_df

,week_birthday,month_birthday,city,country,business_type,cant_users,gmv,gmv_pp,Frubonos,W1,W2,W3,W4,W5,W6,arpu
0,1,1,BAQ,Colombia,Restaurante,58,4657.189060,4577.83024,26.4828,37,38,39,28,22,19,78.928108
1,1,1,BHZ,Brasil,Restaurante,104,6914.578942,6006.37586,42.1049,65,55,56,45,39,34,57.753614
2,1,1,BOG,Colombia,Restaurante,131,9850.504836,8711.74935,60.1160,90,85,87,86,78,62,66.501903
3,1,1,CMX,México,Restaurante,162,12393.732823,11198.43519,71.1444,94,81,85,71,49,42,69.126143
4,1,1,CWB,Brasil,Restaurante,21,1325.132630,1179.32557,10.3157,10,5,7,5,7,5,56.158360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,30,7,CMX,México,Restaurante,22,1560.329262,1538.12247,6.3613,0,0,0,0,0,0,69.914658
277,30,7,CWB,Brasil,Restaurante,20,968.795788,968.79431,13.6842,0,0,0,0,0,0,48.439715
278,30,7,MDE,Colombia,Restaurante,33,1618.961631,1453.80051,9.2690,0,0,0,0,0,0,44.054561
279,30,7,SPO,Brasil,Restaurante,85,8093.362101,7478.87869,74.6037,0,0,0,0,0,0,87.986808


In [21]:
import gspread
gc = gspread.service_account(filename='google_sheets_keys_yeru.json')
worksheet = gc.open('Presupuesto Paid y GMV | ANTONIO').worksheet('Cópia de Costos Paid 2023')
data = worksheet.get_all_values()
df_paid = pd.DataFrame(data[1:], columns=data[0])
df_paid

,ciudad,FB,Google,Total,GMV,%,semana
0,BOG,$721.3,"$3,260.2","$3,981.4","$666,554.0",0.6%,1
1,BAQ,$444.7,$597.4,"$1,042.1","$234,324.5",0.4%,1
2,MDE,$697.0,$476.7,"$1,173.7","$180,288.9",0.7%,1
3,CMX,$381.6,"$2,861.7","$3,243.3","$578,742.8",0.6%,1
4,SPO,"$4,055.1","$2,019.8","$6,074.9","$1,322,839.1",0.5%,1
...,...,...,...,...,...,...,...
227,CMX,$962.2,"$1,512.8","$2,475.01","$794,141.6",0.31%,29
228,SPO,"$2,462.9","$2,196.6","$4,659.47","$2,243,200.5",0.21%,29
229,BHZ,"$1,114.7",$885.7,"$2,000.42","$408,382.2",0.49%,29
230,CWB,$641.2,$363.2,"$1,004.38","$197,651.4",0.51%,29


In [22]:
def remove_usd(value):
    if isinstance(value, str) and '$' in value:
        return value.replace('$', '')
    return value

df_paid['Total'] = df_paid['Total'].apply(remove_usd)
df_paid['GMV'] = df_paid['GMV'].apply(remove_usd)
df_paid.rename(columns={'ciudad': 'city'}, inplace=True)
df_paid.rename(columns={'semana': 'week_birthday'}, inplace=True)
#df_paid['Total'] = df_paid['Total'].str.replace('.', ',').str.replace(',', '')
df_paid['Total'] = df_paid['Total'].str.replace(',', '')
df_paid['Total'] = df_paid['Total'].astype(float)
df_paid['GMV'] = df_paid['GMV'].str.replace(',', '')
df_paid['GMV'] = df_paid['GMV'].astype(float)
df_paid['week_birthday'] = df_paid['week_birthday'].astype(int)
df_paid.rename(columns={'Total': 'Paid'}, inplace=True)
df_paid.rename(columns={'GMV': 'GMV Total'}, inplace=True)
df_paid

,city,FB,Google,Paid,GMV Total,%,week_birthday
0,BOG,$721.3,"$3,260.2",3981.40,666554.0,0.6%,1
1,BAQ,$444.7,$597.4,1042.10,234324.5,0.4%,1
2,MDE,$697.0,$476.7,1173.70,180288.9,0.7%,1
3,CMX,$381.6,"$2,861.7",3243.30,578742.8,0.6%,1
4,SPO,"$4,055.1","$2,019.8",6074.90,1322839.1,0.5%,1
...,...,...,...,...,...,...,...
227,CMX,$962.2,"$1,512.8",2475.01,794141.6,0.31%,29
228,SPO,"$2,462.9","$2,196.6",4659.47,2243200.5,0.21%,29
229,BHZ,"$1,114.7",$885.7,2000.42,408382.2,0.49%,29
230,CWB,$641.2,$363.2,1004.38,197651.4,0.51%,29


In [23]:
grouped_df = pd.merge(grouped_df, df_paid[['city', 'Paid','GMV Total','week_birthday']], on=['week_birthday','city'], how='left')
grouped_df['CAC']=grouped_df['Paid']/grouped_df['cant_users']
ordem_colunas = ['cant_users', 'country', 'city', 'week_birthday', 'month_birthday', 'business_type', 'gmv', 'gmv_pp', 'arpu', 'Paid', 'CAC', 'GMV Total', 'Frubonos','W1','W2','W3','W4','W5','W6']
grouped_df = grouped_df.reindex(columns=ordem_colunas)

In [24]:
grouped_df['Paid'] = grouped_df['Paid'].fillna('')
grouped_df['CAC'] = grouped_df['CAC'].fillna('')
grouped_df['GMV Total'] = grouped_df['GMV Total'].fillna('')
grouped_df

,cant_users,country,city,week_birthday,month_birthday,business_type,gmv,gmv_pp,arpu,Paid,CAC,GMV Total,Frubonos,W1,W2,W3,W4,W5,W6
0,58,Colombia,BAQ,1,1,Restaurante,4657.189060,4577.83024,78.928108,1042.1,17.967241,234324.5,26.4828,37,38,39,28,22,19
1,104,Brasil,BHZ,1,1,Restaurante,6914.578942,6006.37586,57.753614,2407.6,23.15,284404.4,42.1049,65,55,56,45,39,34
2,131,Colombia,BOG,1,1,Restaurante,9850.504836,8711.74935,66.501903,3981.4,30.392366,666554.0,60.1160,90,85,87,86,78,62
3,162,México,CMX,1,1,Restaurante,12393.732823,11198.43519,69.126143,3243.3,20.02037,578742.8,71.1444,94,81,85,71,49,42
4,21,Brasil,CWB,1,1,Restaurante,1325.132630,1179.32557,56.158360,1271.2,60.533333,114495.9,10.3157,10,5,7,5,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,22,México,CMX,30,7,Restaurante,1560.329262,1538.12247,69.914658,,,,6.3613,0,0,0,0,0,0
277,20,Brasil,CWB,30,7,Restaurante,968.795788,968.79431,48.439715,,,,13.6842,0,0,0,0,0,0
278,33,Colombia,MDE,30,7,Restaurante,1618.961631,1453.80051,44.054561,,,,9.2690,0,0,0,0,0,0
279,85,Brasil,SPO,30,7,Restaurante,8093.362101,7478.87869,87.986808,,,,74.6037,0,0,0,0,0,0


In [25]:
content2=[grouped_df.columns.values.tolist()]+grouped_df.values.tolist()
ID_PERFORMANCE_DASH_DATA = '1VlM-Cyd9_dr-VPOko7GccgeaalNc4aq4hnNtWZr5Byg'

google_sheets_methods.clear_content(ID_PERFORMANCE_DASH_DATA,'base2','A:T')
google_sheets_methods.write_content(ID_PERFORMANCE_DASH_DATA,'base2','A:T',content2)
print("New budgets updated")

Range successfully deleted
File successfully updated
New budgets updated


In [26]:
#with pd.ExcelWriter('Base_pp.xlsx') as writer:
 #   hunters.to_excel(writer, sheet_name = 'Experimento', index = False)